# User Quick Start
<sub>Note if you are viewing this on github, you may need to view it on Google Colab [![Open In GoogleColab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Ahuge/sept/blob/release/docs/introduction/non-developer.ipynb)</sub>  

## Introduction
This `User Quick Start` guide tries to demonstrate

- How to get started modifying your own ``SEPT`` templates
- Why you should care and flexibility you can gain from using ``SEPT``


### Installation
Users can easily install ``SEPT`` according to the following steps:

Installing ``SEPT`` from the python packaging index is as simple as executing the following command:
```bash
pip install sept
```

## Getting Started
This tutorial is aimed at someone that is not a developer and instead will be a user of some program that a developer writes that takes advantage of ``SEPT``

For this tutorial, we are going to learn how to write ``SEPT`` templates using the simple application that was created from the [Developer Sample Application](./developer.ipynb#sample-qt-application) tutorial.

The finished product from that tutorial will look similar to the following:

![Finished Qt Application from the Sample Qt Application Developer Tutorial](../images/developer_introduction_sample_qt_application_finished.gif)

This application is a program that will prepare several "Version" entities from your internal `Shotgun <https://shotgunsoftware.com>`_ website. If you haven't worked with Shotgun before, don't worry. You should be able to substitute Shotgun for any production tracking software, and a "Version" entity for any rendered image sequence that can is approved to send back to the movie studio.

There are 2 main components to this application.

On the left side of the dialog you can see the editor window where we can test our ``SEPT`` template.

![Highlighting the template editing and preview on the left side of the dialog](../images/developer_introduction_sample_qt_application_preview.png)

On the right side is the help window that shows all of the ``Token`` and ``Operator`` keywords that you have access to.

![Highlighting the Documentation help page on the right side of the dialog](../images/developer_introduction_sample_qt_application_documentation.png)

## Modifying Templates
The following are some examples of path templates that you may write.  

For simplicity's sake, the examples are going to work off of a single input file, however in reality you may be working with tens or even hundreds of input files at once.  

The quicktime file will have the following information about it
```yaml
project: HeroJourney
sequence: Battle
shot: 001
step: comp
version: 2
extension: mov
```

Within your studio, the filepath looks like `HeroJourney_Battle_001_comp_v1.mov`.

In [ ]:
!pip install sept
from sept import PathTemplateParser
data = {
    "project": "HeroJourney",
    "sequence": "Battle",
    "shot": "001",
    "step": "comp",
    "version": 2,
    "extension": "mov",
}

parser = PathTemplateParser()


### First Example
In this example, your client expects the movie file to no longer have the project code when you deliver it to them.  
For example, we want the filename to be `Battle_001_comp_v2.mov`.  
This means we need to write a custom template to remove the project code.
  
The following code block allows you to write a `SEPT` template and execute it to see the result.

In [ ]:
# Type your SEPT template here:
template_str = "{{sequence}}_{{shot}}_{{step}}_v{{version}}.{{extension}}"

result = parser.parse(template_str, data)
print(result)

#### Breaking It Down
The template above takes the `sequence`, `shot`, and `step` tokens and joins them with an underscore.  
It then adds "_v" and the `version` token to the end before adding the file `extension` token at the end.  
  
To put a `Token` in your template you can place the name of your `Token` between two sets of curly brackets and `SEPT` will write out whatever value is in the `data` with that name.  
```
{{tokenName}}
```
  
You can put any characters you would like outside of the curly brackets and they will be rendered exactly as you have written them.  

## Introduction To Operators
There are times when the client requires naming that cannot be created by just adding tokens together from your Shotgun data.  
In these cases you may need to apply an `Operator` to a `Token` to change it in some way.  
  
`SEPT` provides several common operations out of the box but a developer can write custom ones that may apply better to your specific use case.  
If there is functionality that `SEPT` does not provide out of the box that you think it should, please reach out and let me know what you think it should provide!  

### Using An Operator
To use an `Operator` with your `Token` you need to modify how you write the expression.  
Instead of `{{tokenName}}`, you should instead write `{{operatorName:tokenName}}`.  
The syntax extends the syntax you already have learned by adding an `operatorName` followed by a full colon `:` and then the `tokenName`.

### Lowercase Template Example
In this example, our client has requested that everything in our filename is lowercase.  
Without using an `Operator`, there is no easy way to achieve this, you would need to request that a producer on the show changes the name of the sequence from "Battle" to "battle". If this is at the start of the project, it may not be a huge deal, but as soon as work has started, this becomes nearly impossible to achieve without having to redo work.  

But not to worry! Operators are here!  
To create a filename that looks like `battle_001_comp_v2.mov`, we just need to apply a "lower" `Operator` on the sequence `Token`.  

In [ ]:
# Type your SEPT template here:
template_str = "{{lower:sequence}}_{{shot}}_{{step}}_v{{version}}.{{extension}}"

result = parser.parse(template_str, data)
print(result)

#### Breaking It Down
Luckily for us, we know that all of the shots are numbers and lowercasing a number doesn't change it so all we have to apply `lower` to is the "sequence" `Token`.  
If you compare this to the previous template that we wrote, you will see that the only change is the addition of our `Operator` right at the beginning.  

### Operators With Inputs
There are some advanced `Operator` types that require additional information from you to do their work.  
A good example of one of these would be the `replace` `Operator`. This allows you to find and replace characters in your Token.  
It needs to know what you want to find and what you want to replace it with.
  
These are called "Operator Inputs" and any `Operator` that requires them should provide a description of what it expects and some examples of using it.
  
To set the value of an input, we need to surround it in square brackets directly after the name of our `Operator`.
```
{{operatorName[inputValue]:tokenName}}
```
Some operators may expect multiple input values, the syntax for this is very similar, you just need to separate the input values with a comma.
```
{{operatorName[inputValue1,inputValue2]:tokenName}}
```
  
Below is the description from `replace`:

```
replace Operator
 Operator Inputs

  Find String: The characters that you want to search for and replace 
    Required: True

  Replace String: The characters that you want to replace the "Find String" with. 
    Required: True


The replace Operator allows you to find and replace characters in your Token. 
Examples (name = "alex"): 
 {{replace[ex,an]:name}}   -> "alan" 
 {{replace[kite,dog:name}} -> "alex"
```

#### Replace Sequence Example
In this example, our client has renamed the "Battle" sequence and decided that it is now called "Conflict".  
This messes us up because we have been working with it as "Battle" and we don't want to redo any work.

Not to worry! Operators are here!  
To create a filename that looks like `Conflict_001_comp_v2.mov`, we just need to apply a "replace" `Operator` on the sequence `Token`.  

In [ ]:
# Type your SEPT template here:
template_str = "{{replace[Battle,Conflict]:sequence}}_{{shot}}_{{step}}_v{{version}}.{{extension}}"

result = parser.parse(template_str, data)
print(result)

#### Breaking It Down
Because we know that our sequence is called "Battle", we can search for the word "Battle" and replace it with "Conflict".  
This is preferable to just writing the word "Conflict" in there because now our template will work even if the input file is from a different sequence because we only want to replace the "Battle" sequence.  

This (again) is an extension of our syntax, compared to our "lower" example, you can see we have added `[Battle,Conflict]` in our template.  
If we refer to the "replace" documentation above we can see that "replace" takes two inputs, `Find String` and `Replace String`.  
In our example we have set the `Find String` equal to "Battle" and the `Replace String` equal to "Conflict". This means that any time it finds "Battle" as the sequence, it will replace it with "Conflict".  
   

### Nested Operators
There may be certain times when you need to apply more than one `Operator` to a `Token` in order to get exactly what you want.  
  
`SEPT` fully supports this by nesting an `Operator` within another `Operator`.  
The syntax for this should be an extension of everything you have learned already. You can take an entire Token Expression and use it as the `Token` value for a separate Token Expression. This allows you to apply more than one `Operator` to a `Token`.  
```
{{operatorName2:{{operatorName1:tokenName}}}}
```
  
`SEPT` will apply each `Operator` one at a time inside out.
  

#### Nested Operator Example
In this example we need to return only the first 4 characters from our sequence and then make sure that they are all in uppercase.  
This will introduce you to two new operators that we need to use to achieve our goal.  

The first is the opposite to the "lower" `Operator` that we saw earlier, "upper", and the second is "substr" which allows us to return a subset of the `Token`.  
"upper" doesn't take any inputs and "substr" takes a `Start Location` and optionally a `End Location`.  
  
"substr" is a bit special in that it will only accept certain values as inputs, it takes numbers for the location in the `Token` as well as "start" and "end".  

To create a filename that looks like `BATT_001_comp_v2.mov`, we can use the following expression.  

In [ ]:
# Type your SEPT template here:
template_str = "{{substr[start,4]:{{upper:sequence}}}}_{{shot}}_{{step}}_v{{version}}.{{extension}}"

result = parser.parse(template_str, data)
print(result)

#### Breaking It Down
In the above example, we are applying the "upper" and then the "substr", in this specific case it doesn't matter which one happens first but it is important to keep in mind which one is being applied first.  
We are passing the "substr" `Operator` two inputs, the first is "start" and the second is "4".  
The value "start" will always equal "0" but for people that are not familiar with zero based index collections "start" is clearer.  

## Conclusion
You've reached the end of our interactive tutorial, this should have taught you the basics of writing a `SEPT` template.  

 - You learned how to write a `Token`
 - Apply an `Operator` to a `Token`
 - Customize how the `Operator` works by passing it input values
 - Apply more than one `Operator` to a `Token` by nesting the operators.
  
_____
  
If you are interested in a more technical understanding of how `SEPT` works and how you can customize it to work better for your company, you should check out the [Developer Introduction](./developer.ipynb)
